## Stappenplan en verdeling

1. aanmaken github
2. data preprocessing (verwijder duplicates and NaN)
3. bepaal de relevante features van de descriptors via een PCA
4. wat wordt test en wat wordt train
5. aanmaken model voor het maken van voorspellingen
    5.1 uitproberen welke methodes er toepasbaar zijn (random forest, k-NN)
    5.2 bepalen welke methode het beste is
6. verslag en presentatie 

1. Richard
2. Luuk, Richard
3. Robin, Tijn, Rik & Yael
4. 
5. Julien & Branco
6. 

## 5.1 Onderzoek over random forest methode
The random forest algorithm is an extension of the bagging method as it utilizes both bagging and feature randomness to create an uncorrelated forest of decision trees. While decision trees consider all the possible feature splits, random forests only select a subset of those features.

Random forest algorithms have three main hyperparameters, which need to be set before training. These include node size, the number of trees, and the number of features sampled. From there, the random forest classifier can be used to solve for regression or classification problems.

> Dit betekent dat alle descriptors onze features zijn waarvan per tree een random keuze wordt gemaakt. 
> The nodesize parameter specifies the minimum number of observations in a terminal node. Setting it lower leads to trees with a larger depth which means that more splits are performed until the terminal nodes.

The random forest algorithm is made up of a collection of decision trees, and each tree in the ensemble is comprised of a data sample drawn from a training set with replacement, called the bootstrap sample. Of that training sample, one-third of it is set aside as test data, known as the out-of-bag (oob) sample, which we’ll come back to later. Another instance of randomness is then injected through feature bagging, adding more diversity to the dataset and reducing the correlation among decision trees. Depending on the type of problem, the determination of the prediction will vary.

> Dit geeft dus aan dat onze verschillende moleculen niet allemaal tegelijk moeten worden gebruikt maar in secties. De grote van deze sectie is denk ik voor onszelf te bepalen. 

In [3]:
#Useful functions:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
#Hiermee importeren we de RandomForestClassifier functie.

# Load the dataset
data = load_iris()
X = data.data
y = data.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier()
#RandomForestClassifier() kan veel parameters gebruiken. De belangrijkste hiervan is n_estimators. Deze parameter bepaalt de hoeveelheid decision trees. Sommige parameters, zoals max_leaf_nodes, welke de maximale hoeveelheid verschillende ‘kruispunten’ in elke decision tree bepaalt, kunnen worden gebruikt om het model te optimaliseren.

rf.fit(X_train, y_train)
#Deze functie fit het model op onze data. Xtrain en Ytrain zijn in dit geval training data. Dat is dus die file tested_molecules.csv. 

Ypredicted = rf.predict(X_test)
#Nadat het model is gefit, kunnen met deze functie de waardes voor onze test data berekend worden.

ChatGPT test

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.datasets import load_iris

# Load the dataset
data = load_iris()
X = data.data
y = data.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(report)

Accuracy: 1.0
Confusion Matrix:
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



## try random forest with our data

In [6]:

data=pd.read_csv('cleaned_descriptor_data',index_col=0)
#data=pd.read_csv('cleaned_fingerprint_data',index_col=0)

data.head()



,BertzCT,NOCount,Kappa1,HeavyAtomMolWt,BCUT2D_LOGPLOW,fr_amide,NumValenceElectrons,HeavyAtomCount,Kappa2,HallKierAlpha,...,fr_bicyclic,fr_ester,PEOE_VSA11,fr_NH2,fr_piperdine,fr_oxazole,fr_pyridine,fr_priamide,ERK2_inhibition,PKM2_inhibition
SMILES,,,,,,,,,,,,,,,,,,,,,
C=C(C)c1nc(N)nc(N)n1,272.774159,5,7.456952,142.101,-2.047656,0,58,11,2.531038,-1.65,...,0,0,5.824404,2,0,0,0,0,0,0
C=C(Cl)COc1ccc2c(C)cc(=O)oc2c1,628.087447,3,11.736876,239.593,-1.991714,0,88,17,4.634332,-1.74,...,1,0,0.000000,0,0,0,0,0,0,0
C=CCNC(=O)CCCC(=O)NCC=C,212.742421,4,13.420000,192.133,-2.250322,2,84,15,8.993915,-1.58,...,0,0,0.000000,0,0,0,0,0,0,0
C=CCOn1c(=O)c(C)[n+]([O-])c2ccccc21,631.418725,5,11.193326,220.143,-2.432024,0,88,17,4.288212,-2.30,...,1,0,5.516701,0,0,0,0,0,0,0
C=CCn1cc(Cl)c(=O)n(CC=C)c1=O,499.124344,4,11.524349,215.575,-2.388420,0,80,15,4.794989,-1.55,...,0,0,0.000000,0,0,0,0,0,0,0


In [25]:

X = data.drop(columns='ERK2_inhibition').drop(columns='PKM2_inhibition').copy()
y = data['ERK2_inhibition']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# Initialize the Random Forest Classifier
rf = RandomForestClassifier(n_estimators=1100)

# Train the model
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(report)
print(y_pred)

Accuracy: 0.9642857142857143
Confusion Matrix:
[[216   0]
 [  8   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       216
           1       0.00      0.00      0.00         8

    accuracy                           0.96       224
   macro avg       0.48      0.50      0.49       224
weighted avg       0.93      0.96      0.95       224

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Test/train with weight

In [30]:
X = data.drop(columns='ERK2_inhibition').drop(columns='PKM2_inhibition').copy()
y = data['ERK2_inhibition']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)#, random_state=40) # random state is om elke x zelfde resulaten te krijgen, handig om te kijken of bepaalde tezt/train invloed heeft

class_weights = {0: 0.5, 1: 1} 

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(n_estimators=1100, class_weight=class_weights)

# Train the model
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(report)
print(y_pred)

Accuracy: 0.9375
Confusion Matrix:
[[210   0]
 [ 14   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       210
           1       0.00      0.00      0.00        14

    accuracy                           0.94       224
   macro avg       0.47      0.50      0.48       224
weighted avg       0.88      0.94      0.91       224

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Test/train with equal ration 0 and 1

In [53]:

# Load the dataset
data = pd.read_csv('cleaned_descriptor_data', index_col=0)

# Define features (X) and target (y)
X = data.drop(columns=['ERK2_inhibition', 'PKM2_inhibition']).copy()
y = data['ERK2_inhibition']

# Split the data into training and testing sets while preserving the ratio of 0 and 1 inhibitors
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Calculate the ratio of 0 and 1 inhibitors in the training set
ratio_train = y_train.value_counts(normalize=True)

# Calculate the ratio of 0 and 1 inhibitors in the testing set
ratio_test = y_test.value_counts(normalize=True)

print("Ratio of 0 and 1 inhibitors in the training set:")
print(ratio_train)
print("\nRatio of 0 and 1 inhibitors in the testing set:")
print(ratio_test)

# Initialize the Random Forest Classifier with appropriate class weights
# class_weights = {0: 1, 1: 1}  # You can adjust the weights based on the class distribution
rf = RandomForestClassifier(n_estimators=1100)#, class_weight=class_weights)

# Train the model
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"\nAccuracy: {accuracy}")
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(report)
print(y_pred)

Ratio of 0 and 1 inhibitors in the training set:
0    0.954036
1    0.045964
Name: ERK2_inhibition, dtype: float64

Ratio of 0 and 1 inhibitors in the testing set:
0    0.955357
1    0.044643
Name: ERK2_inhibition, dtype: float64

Accuracy: 0.9553571428571429
Confusion Matrix:
[[214   0]
 [ 10   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       214
           1       0.00      0.00      0.00        10

    accuracy                           0.96       224
   macro avg       0.48      0.50      0.49       224
weighted avg       0.91      0.96      0.93       224

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Over and undersampling test/train

In [54]:

# Load the dataset
data = pd.read_csv('cleaned_descriptor_data', index_col=0)

# Define features (X) and target (y)
X = data.drop(columns=['ERK2_inhibition', 'PKM2_inhibition']).copy()
y = data['ERK2_inhibition']

# Split the data into training and testing sets while preserving the ratio of 0 and 1 inhibitors
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Manual Oversampling
class_1_count = y_train.value_counts()[1]
class_0_count = y_train.value_counts()[0]

# Get indices of class 1
class_1_indices = np.where(y_train == 1)[0]

# Calculate number of samples needed to balance the classes
additional_samples_needed = class_0_count - class_1_count

# Randomly duplicate class 1 samples
additional_class_1_indices = np.random.choice(class_1_indices, size=additional_samples_needed, replace=True)

# Append duplicated samples to the training set
X_train_over = np.vstack([X_train, X_train.iloc[additional_class_1_indices]])
y_train_over = np.hstack([y_train, y_train.iloc[additional_class_1_indices]])

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(n_estimators=1100, random_state=42)

# Train the model on the oversampled data
rf.fit(X_train_over, y_train_over)

# Make predictions
y_pred_over = rf.predict(X_test)

# Evaluate the model
accuracy_over = accuracy_score(y_test, y_pred_over)
cm_over = confusion_matrix(y_test, y_pred_over)
report_over = classification_report(y_test, y_pred_over)

print("\nManual Oversampling Results:")
print(f"Accuracy: {accuracy_over}")
print("Confusion Matrix:")
print(cm_over)
print("Classification Report:")
print(report_over)
print(y_pred)
# Manual Undersampling
class_0_indices = np.where(y_train == 0)[0]

# Calculate number of samples to keep to balance the classes
samples_to_keep = class_1_count

# Randomly select a subset of class 0 samples
undersample_class_0_indices = np.random.choice(class_0_indices, size=samples_to_keep, replace=False)

# Append selected samples to the training set
X_train_under = np.vstack([X_train.iloc[undersample_class_0_indices], X_train[y_train == 1]])
y_train_under = np.hstack([y_train.iloc[undersample_class_0_indices], y_train[y_train == 1]])

# Train the model on the undersampled data
rf.fit(X_train_under, y_train_under)

# Make predictions
y_pred_under = rf.predict(X_test)

# Evaluate the model
accuracy_under = accuracy_score(y_test, y_pred_under)
cm_under = confusion_matrix(y_test, y_pred_under)
report_under = classification_report(y_test, y_pred_under)

print("\nManual Undersampling Results:")
print(f"Accuracy: {accuracy_under}")
print("Confusion Matrix:")
print(cm_under)
print("Classification Report:")
print(report_under)
print(y_pred)


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming data is already loaded and preprocessed
X = data.drop(columns=['ERK2_inhibition', 'PKM2_inhibition']).copy()
y = data['ERK2_inhibition']

# Split the dataset into training and testing sets with stratification to maintain the class ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Combine X_train and y_train for resampling
train_data = pd.concat([X_train, y_train], axis=1)

# Separate the minority and majority classes
minority_class = train_data[train_data['ERK2_inhibition'] == 1]
majority_class = train_data[train_data['ERK2_inhibition'] == 0]

# Undersample the majority class
undersampled_majority = majority_class.sample(n=len(minority_class), random_state=42)

# Oversample the minority class
oversampled_minority = minority_class.sample(n=len(majority_class), replace=True, random_state=42)

# Combine the undersampled majority and oversampled minority
resampled_train_data = pd.concat([undersampled_majority, oversampled_minority])

# Separate features and target variable from the resampled dataset
X_train_resampled = resampled_train_data.drop(columns='ERK2_inhibition')
y_train_resampled = resampled_train_data['ERK2_inhibition']

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(n_estimators=1100, random_state=42)

# Train the model
rf.fit(X_train_resampled, y_train_resampled)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(report)



Manual Oversampling Results:
Accuracy: 0.9508928571428571
Confusion Matrix:
[[213   1]
 [ 10   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.97       214
           1       0.00      0.00      0.00        10

    accuracy                           0.95       224
   macro avg       0.48      0.50      0.49       224
weighted avg       0.91      0.95      0.93       224

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]

Manual Undersampling Results:
Accuracy: 0.6116071428571429
Confusion Matrix:
[[129  85]
 [  2

For now, the random forest just predicts every molecule to have no inhibition and this gives a pretty high accuracy. We will probably need a workaround this

## Try kNN model for own data

In [8]:
# Import packages needed
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [9]:
# Load the dataset
data_knn = pd.read_csv('cleaned_descriptor_data',index_col=0)
data.head()

,BertzCT,NOCount,Kappa1,HeavyAtomMolWt,BCUT2D_LOGPLOW,fr_amide,NumValenceElectrons,HeavyAtomCount,Kappa2,HallKierAlpha,...,fr_bicyclic,fr_ester,PEOE_VSA11,fr_NH2,fr_piperdine,fr_oxazole,fr_pyridine,fr_priamide,ERK2_inhibition,PKM2_inhibition
SMILES,,,,,,,,,,,,,,,,,,,,,
C=C(C)c1nc(N)nc(N)n1,272.774159,5,7.456952,142.101,-2.047656,0,58,11,2.531038,-1.65,...,0,0,5.824404,2,0,0,0,0,0,0
C=C(Cl)COc1ccc2c(C)cc(=O)oc2c1,628.087447,3,11.736876,239.593,-1.991714,0,88,17,4.634332,-1.74,...,1,0,0.000000,0,0,0,0,0,0,0
C=CCNC(=O)CCCC(=O)NCC=C,212.742421,4,13.420000,192.133,-2.250322,2,84,15,8.993915,-1.58,...,0,0,0.000000,0,0,0,0,0,0,0
C=CCOn1c(=O)c(C)[n+]([O-])c2ccccc21,631.418725,5,11.193326,220.143,-2.432024,0,88,17,4.288212,-2.30,...,1,0,5.516701,0,0,0,0,0,0,0
C=CCn1cc(Cl)c(=O)n(CC=C)c1=O,499.124344,4,11.524349,215.575,-2.388420,0,80,15,4.794989,-1.55,...,0,0,0.000000,0,0,0,0,0,0,0


In [10]:
# Split data into predictors and outcomes
X_knn = data_knn.drop(columns=['ERK2_inhibition','PKM2_inhibition']).copy()
y_knn = data_knn['ERK2_inhibition']

# Create training and test sets
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X_knn, y_knn, test_size=0.2)

# Train the model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_knn, y_train_knn)

# Make predictions of test set
y_pred_knn = knn.predict(X_test_knn)

# Assess model performances
accuracy_knn = accuracy_score(y_test_knn, y_pred_knn)
cm_knn = confusion_matrix(y_test_knn, y_pred_knn)
report_knn = classification_report(y_test_knn, y_pred_knn)

print(f"Accuracy: {accuracy_knn}")
print("Confusion Matrix:")
print(cm_knn)
print("Classification Report:")
print(report_knn)
print(y_pred_knn)



Accuracy: 0.9642857142857143
Confusion Matrix:
[[216   0]
 [  8   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       216
           1       0.00      0.00      0.00         8

    accuracy                           0.96       224
   macro avg       0.48      0.50      0.49       224
weighted avg       0.93      0.96      0.95       224

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Try kNN with automated parameter tuning

In [11]:
parameters = {'n_neighbors' : range(1,30), 'weights' : ['uniform', 'distance']}
gridsearch_knn = GridSearchCV(estimator= KNeighborsClassifier(),
                              param_grid=parameters,
                              scoring='accuracy',
                              return_train_score=True)
gridsearch_knn.fit(X_train_knn, y_train_knn)

# Optimal model parameters
gridsearch_knn.best_params_

{'n_neighbors': 4, 'weights': 'uniform'}

In [11]:
positive=data[data['ERK2_inhibition']==1]+data[data['PKM2_inhibition']==1]
positive.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78 entries, C/C=C/c1ccc(OCCCCN2C(=O)NC(C)(C)C2=O)c(OC)c1 to c1ccc2c(c1)ncn2Cc1nnc2sc(-c3ccncc3)nn12
Columns: 194 entries, Bit_696 to PKM2_inhibition
dtypes: float64(194)
memory usage: 118.8+ KB
